# Smarticle Simulation

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preliminaries

In [9]:
import pybullet as p
import time
import pybullet_data
import smarticlesimulation as sim
import numpy as np
import keyboard
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib widget

############
# Load paths
smarticle_path = '../urdf/smarticle.urdf'
ring_path = '../urdf/ring.urdf'
data_path = '../data/'

############
# Parameters
viz = False
num_trials = 10
rand_len = 0*60 # random gait time
sim_len = 8*60  # sim time
rand_steps = sim.time_to_steps(rand_len)
sim_steps = sim.time_to_steps(sim_len)
Ns = 3 # number of smarticles
width = 0.032 # placement spacing
th = np.pi/2 # inital orientation

## Set Up

In [10]:
#############
# Environment
if viz:
    physicsClient = p.connect(p.GUI)
else:
    physicsClient = p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
p.setGravity(0,0,-9.81)
planeId = p.loadURDF("plane.urdf")
ringID = p.loadURDF(ring_path, basePosition = [0,0,0])
ringConstraintID = p.createConstraint(ringID,-1,0,-1,p.JOINT_FIXED,[0,0,0],[0,0,0],[0,0,0])

#################
# Smarticle gaits
R = [-np.pi/2,-np.pi/2,np.pi/2,np.pi/2,np.pi/2,np.pi/2,-np.pi/2,-np.pi/2]
L = [np.pi/2,np.pi/2,np.pi/2,np.pi/2,-np.pi/2,-np.pi/2,-np.pi/2,-np.pi/2]
gaits = [[L,R],[L,R],[L,R]]
gait_period = sim.time_to_steps(0.2)
smarticles = sim.load_smarticles(Ns, smarticle_path, gaits, gait_period, 0, width, th)

# Simulation

In [11]:
data = []
tic = time.time()
for i in range(num_trials):
    print("Trial Num: "+str(i+1))
    
    ###############
    # Randomization
    for j in range(rand_steps):
        data.append(sim.get_state(smarticles))
        p.stepSimulation()
        for s in smarticles:
            if j % gait_period == s.gait_phase:
                s.move_random_corners()
            if keyboard.is_pressed('q'):
                break
        if keyboard.is_pressed('q'):
            break
    
    ##########
    # Main run
    for j in range(sim_steps):
        if viz:
            time.sleep(1/240.)
        data.append(sim.get_state(smarticles,i+1))
        p.stepSimulation()
        for s in smarticles:
            if j % gait_period == s.gait_phase:
                s.motor_step()
            if keyboard.is_pressed('q'):
                break
        if keyboard.is_pressed('q'):
            break

p.disconnect()
toc = time.time()
print("Time Elapsed: "+str(np.round(toc-tic,1))+"s")

Trial Num: 1
Trial Num: 2
Trial Num: 3
Trial Num: 4
Trial Num: 5
Trial Num: 6
Trial Num: 7
Trial Num: 8
Trial Num: 9
Trial Num: 10
Time Elapsed: 1950.9s


## Subsampling and Saving

In [12]:
DATE = "2020-10-13"
F = 2
skip = 1
data = np.copy(data[::skip])
filename = DATE+"_sim_randinit_"+str(num_trials)+"_240Hz_file_"+str(F)+".csv"
np.savetxt(data_path+filename, data, delimiter=",")

## Check Plot

In [7]:
skip = 10
norm_len = 1./12. 
t_data = np.linspace(0,data.shape[0]/240.,data.shape[0])
x_unscaled = data[:,:3*Ns:3]
y_unscaled = data[:,1:3*Ns:3]
theta = data[:,2:3*Ns:3]
centroid = np.array([np.mean(x_unscaled,axis=1),np.mean(y_unscaled,axis=1)])
origin = np.array([np.mean(centroid[0]),np.mean(centroid[1])])

# Scaled data
x = (x_unscaled-origin[0])/norm_len
y = (y_unscaled-origin[1])/norm_len

# Check trajectory
plt.figure()
plt.subplot(3,1,1)
plt.title('Simulated Smarticle Trajectories')
plt.scatter(t_data,x.T[0],s=1.5)
plt.scatter(t_data,x.T[1],s=1.5)
plt.scatter(t_data,x.T[2],s=1.5)
plt.xlim([t_data[0], t_data[-1]])
plt.ylabel(r'$x$')
plt.grid(True)

plt.subplot(3,1,2)
plt.scatter(t_data,y.T[0],s=1.5)
plt.scatter(t_data,y.T[1],s=1.5)
plt.scatter(t_data,y.T[2],s=1.5)
plt.xlim([t_data[0], t_data[-1]])
plt.ylabel(r'$y$')
plt.grid(True)

plt.subplot(3,1,3)
plt.scatter(t_data,theta.T[0],s=1.5)
plt.scatter(t_data,theta.T[1],s=1.5)
plt.scatter(t_data,theta.T[2],s=1.5)
plt.xlim([t_data[0], t_data[-1]])
plt.ylabel(r'$\theta$')
plt.xlabel('Time (s)')
plt.grid(True)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.close('all')